In [1]:


import pandas as pd
from wbe_odm.odm_mappers import mcgill_mapper
from config import *

import re

In [2]:
qc_lab = mcgill_mapper.McGillMapper()
qc_lab.read(QC_VIRUS_DATA, STATIC_DATA, QC_VIRUS_SHEET_NAME, QC_VIRUS_LAB)  # noqa

In [6]:
xl = pd.ExcelFile(QC_VIRUS_DATA)
val_sheets = [n for n in xl.sheet_names if 'Compil' in n and 'int' in n]
val_sheets

['QC_Compil_INSTITUTIONS (int)',
 'QC_Compil_RESEAU (int)',
 'QC_Compil_STEP (int)']

In [7]:
sheet = val_sheets[-1]
print(sheet)
sheet_df = xl.parse(sheet, header=0, sorted=False, index_col=0)

QC_Compil_STEP (int)


In [8]:
sheet_cols = [str(col) for col in sheet_df.columns]
pos_of_cols_w_headers = []
idx_col_pos = 0

for i, col in enumerate(sheet_cols):
    if i==idx_col_pos:
        continue
    if 'Unnamed' not in col:
        pos_of_cols_w_headers.append(i+1)

last_sheet_col = len(sheet_cols)
pos_of_cols_w_headers.append(last_sheet_col)
print(pos_of_cols_w_headers)

xl_start_cols = []
xl_end_cols = []

pos_of_last_item = len(pos_of_cols_w_headers) - 1
for i in range(len(pos_of_cols_w_headers.copy())):
    if i == pos_of_last_item:
        #This is the end of the last df, so stop
        break

    start_pos = pos_of_cols_w_headers[i]
    
    if i == pos_of_last_item-1:
        end_pos = pos_of_cols_w_headers[i+1]
        
    else:
        end_pos = pos_of_cols_w_headers[i+1] -1


    start_idx = mcgill_mapper.excel_style(start_pos+1)
    end_idx = mcgill_mapper.excel_style(end_pos+1)
    xl_start_cols.append(start_idx)
    xl_end_cols.append(end_idx)

sites_list = sheet_df.columns.to_list()
sites_list = [i for i in sites_list if "Unnamed" not in i]

type_codes = sheet_df.iloc[1].dropna().to_list()
types = type_codes[::3]
site_info = type_codes[1::3]
label_ids = type_codes[2::3]
last_dates = sheet_df.iloc[2].dropna().to_list()
temp_dates =[]
for item in last_dates:
    try:
        item = pd.to_datetime(item)
        temp_dates.append(item)
    except Exception:
        continue
last_dates = temp_dates
sites = []
for item in label_ids:
    split = item.split("_")[0:2]
    site = "_".join(split).lower()
    sites.append(site)
sites_list, types, site_info, sites, label_ids, last_dates, xl_start_cols, xl_end_cols

[4, 15, 23]


(['Fin du Réseau - Intercepteur Est  (Site 1)',
  'Fin du Réseau - Intercepteur Ouest  (Site 2)'],
 ['cp', 'cp'],
 ['01-Int_East', '02-Int_West'],
 ['qc_01', 'qc_02'],
 ['QC_01_cpFP24h_rawWW', 'QC_02_cpFP24h_rawWW'],
 [Timestamp('2021-05-16 00:00:00'), Timestamp('2021-05-16 00:00:00')],
 ['E', 'P'],
 ['O', 'X'])

In [31]:
new_col_names = sheet_df.iloc[3]
cut_df = sheet_df.iloc[4:].copy()
cut_df.columns = new_col_names
cut_df.index = pd.to_datetime(cut_df.index)

df_idx = 0
reread_df = xl.parse(sheet, header=4, usecols=f"{xl_start_cols[df_idx]}:{xl_end_cols[df_idx]}")
reread_idx = pd.to_datetime(xl.parse(sheet, header=4, usecols="A", squeeze=True)).dt.strftime("%Y-%m-%d")
reread_df.set_index(reread_idx, inplace=True)
reread_df.index = pd.to_datetime(reread_df.index)

cols = reread_df.columns
renamed_cols = {}
for col in cols:
    new_col = col
    if re.match(".*\.[0-9]", col):
        if 'rejected' in col.lower():
            continue
        dot_idx = col.find(".")
        new_col = col[0:dot_idx]
    
    renamed_cols[col] = new_col
reread_df.rename(columns = renamed_cols, inplace=True) 

cols_to_keep = ["BRSV (%rec)","Rejected by", "PMMV (gc/ml)","Rejected by.1", "SARS (gc/ml)", "Rejected by.2", "Quality Note"]
reread_df = reread_df[cols_to_keep]

In [66]:
samples = qc_lab.sample
ww = qc_lab.ww_measure

charac = {
    "BRSV (%rec)": {
        "unit": "pctRecovery",
        "type": "nBRSV",
    },
    "PMMV (gc/ml)": {
        "unit": "gc/ml",
        "type": "nPMMoV",
    },
    "SARS (gc/ml)": {
        "unit": "gc/ml",
        "type": "covN2",
    }
}
copied = reread_df.copy()

# Prepping date columns
samples["dateTimeEnd"] = pd.to_datetime(samples["dateTimeEnd"]) 
samples["dateTime"] = pd.to_datetime(samples["dateTime"]) 

sample_type_filt = samples["type"].str.contains(types[df_idx])
sample_sites_filt = samples["siteID"].str.lower().str.contains(sites[df_idx])
for i, row in copied.iterrows():
    sample_date_filt1 = samples["dateTimeEnd"].dt.date == pd.to_datetime(row.name).date()
    sample_date_filt2 = samples["dateTime"].dt.date == pd.to_datetime(row.name).date()
    sample_date_filt = sample_date_filt1 | sample_date_filt2
    sample_tot_filt = sample_date_filt & sample_sites_filt & sample_type_filt

    samples.loc[sample_tot_filt, ["qualityFlag", "notes"]] = [True, row["Quality Note"]]

    sample_list = samples.loc[sample_tot_filt, "sampleID"].drop_duplicates().to_list()

    for col, wwm_info in charac.items():
        ww_type_filt = ww["type"].str.lower().str.contains(wwm_info["type"])
        ww_unit_filt = ww["unit"].str.lower().str.contains(wwm_info["unit"])
        ww_sample_filt = ww["sampleID"].isin(sample_list)
        ww_tot_filt = ww_type_filt & ww_unit_filt & ww_sample_filt
    
        ww.loc[ww_tot_filt, ["qualityFlag", "notes"]] = [True, row["Quality Note"]]




In [62]:
ww["type"].str.lower()

757       wqph
758       wqph
759       wqph
760       wqph
761       wqph
         ...  
12086    covn2
12087    covn2
12088    covn2
12089    covn2
12090    covn2
Name: type, Length: 6439, dtype: object

In [3]:
sheetname = "QC_COMPIL_STEP (int)"

def find_df_borders(sheet_cols):
    for i, col in enumerate(sheet_cols):
        if i==idx_col_pos:
            continue
        if 'Unnamed' not in col:
            pos_of_cols_w_headers.append(i+1)
    last_sheet_col = len(sheet_cols)
    pos_of_cols_w_headers.append(last_sheet_col)
    print(pos_of_cols_w_headers)

    xl_start_cols = []
    xl_end_cols = []

    pos_of_last_item = len(pos_of_cols_w_headers) - 1
    for i in range(len(pos_of_cols_w_headers.copy())):
        if i == pos_of_last_item:
            #This is the end of the last df, so stop
            break

        start_pos = pos_of_cols_w_headers[i]
        
        if i == pos_of_last_item-1:
            end_pos = pos_of_cols_w_headers[i+1]
            
        else:
            end_pos = pos_of_cols_w_headers[i+1] -1


        start_idx = mcgill_mapper.excel_style(start_pos+1)
        end_idx = mcgill_mapper.excel_style(end_pos+1)
        xl_start_cols.append(start_idx)
        xl_end_cols.append(end_idx)
    return xl_start_cols, xl_end_cols


def get_type_codes(sheet_df):
    return sheet_df.iloc[1].dropna().to_list()


def get_sample_type(type_codes):
    return type_codes[::3]


def get_last_dates(sheet_df):
    dates = sheet_df.iloc[2].dropna().to_list()
    temp_dates =[]
    for item in dates:
        try:
            item = pd.to_datetime(item)
            temp_dates.append(item)
        except Exception:
            continue
    return temp_dates


def get_label_ids(type_codes):
    return type_codes[2::3]


def get_site_ids(label_ids):
    sites = []
    for item in label_ids:
        split = item.split("_")[0:2]
        site = "_".join(split).lower()
        sites.append(site)
    return sites

def get_values_df(path, sheet_name, start, end, header_row_pos):
    return pd.read_excel(
            path,
            sheet_name=sheet_name,
            header=header_row_pos,
            usecols =f"{start}:{end}"
            )

def get_index_series(path, sheet_name, idx_col):
    idx_series = pd.read_excel(
            path,
            sheet_name=sheet_name,
            usecols = idx_col,
            squeeze=True
        )
    idx_series = pd.to_datetime(idx_series).dt.strftime("%Y-%m-%d")
    return pd.to_datetime(idx_series)
    
def clean_names(df):
    cols = df.columns
    renamed_cols = {}
    for col in cols:
        new_col = col
        if re.match(".*\.[0-9]", col):
            if 'rejected' in col.lower():
                continue
            dot_idx = col.find(".")
            new_col = col[0:dot_idx]
    
    renamed_cols[col] = new_col
    return df.rename(columns = renamed_cols) 

def extract_dfs(path, sheet_name, idx_col_pos=0, header_row_pos=4):
    sheet_df = pd.read_excel(path, sheet_name=sheet_name, header=0, sorted=False, index_col=0)
    sheet_cols = [str(col) for col in sheet_df.columns]
    start_borders, end_borders = find_df_borders(sheet_cols)
    idx_col = mcgill_mapper.excel_style(idx_col_pos+1)
    
    dfs = []
    for start, end in zip(start_borders, end_borders):
        vals = get_values_df(path, sheet_name, start, end, header_row_pos)
        idx = get_index_series(path, sheet_name, idx_col)
        df = vals.set_index(idx)
        df = clean_names(df)
        cols_to_keep = ["BRSV (%rec)","Rejected by", "PMMV (gc/ml)","Rejected by.1", "SARS (gc/ml)", "Rejected by.2", "Quality Note"]
        df = df[cols_to_keep]
        dfs.append(df)
    return sheet_df, dfs


def parse_dates(df):
    for col in df.columns:
        if 'dateTime' in col:
            df[col] = pd.to_datetime(col)
    return df


def apply_quality_checks(mapper,  v_df, last_date, site_id, sample_type):
    charac = {
        "BRSV (%rec)": {
            "unit": "pctRecovery",
            "type": "nBRSV",
        },
        "PMMV (gc/ml)": {
            "unit": "gc/ml",
            "type": "nPMMoV",
        },
        "SARS (gc/ml)": {
            "unit": "gc/ml",
            "type": "covN2",
        }
    }
    
    samples = mapper.sample
    samples = parse_dates(samples)
    ww = mapper.ww_measure
    
    sample_type_filt = samples["type"].str.contains(sample_type)
    sample_sites_filt = samples["siteID"].str.lower().str.contains(site_id)
    for _, row in v_df.iterrows():
        sample_date_filt1 = samples["dateTimeEnd"].dt.date == pd.to_datetime(row.name).date()
        sample_date_filt2 = samples["dateTime"].dt.date == pd.to_datetime(row.name).date()
        sample_date_filt = sample_date_filt1 | sample_date_filt2
        sample_tot_filt = sample_date_filt & sample_sites_filt & sample_type_filt

        samples.loc[sample_tot_filt, ["qualityFlag", "notes"]] = [True, row["Quality Note"]]

        sample_list = samples.loc[sample_tot_filt, "sampleID"].drop_duplicates().to_list()

        for col, wwm_info in charac.items():
            ww_type_filt = ww["type"].str.lower().str.contains(wwm_info["type"])
            ww_unit_filt = ww["unit"].str.lower().str.contains(wwm_info["unit"])
            ww_sample_filt = ww["sampleID"].isin(sample_list)
            ww_tot_filt = ww_type_filt & ww_unit_filt & ww_sample_filt
        
            ww.loc[ww_tot_filt, ["qualityFlag", "notes"]] = [True, row["Quality Note"]]

    if 'grb' in sample_type:
        sample_last_date_filt = samples.loc[samples['dateTime'] > last_date]
    else:
        sample_last_date_filt = samples.loc[samples['dateTimeEnd'] > last_date]
    
    unchecked_filt = sample_type_filt & sample_sites_filt & sample_last_date_filt
    samples.loc[unchecked_filt, ["qualityFlag", "notes"]] = [True, "Unchecked"]
    
    unchecked_sample_ids = samples.loc[unchecked_filt, "sampleID"].drop_duplicates().to_list()

    ww.loc[ww['sampleID'].isin(unchecked_sample_ids), ["qualityFlag", "notes"]] = [True, "Unchecked"]
    
    mapper.sample = samples
    mapper.ww_measure = ww
    return mapper

def read_validation(mapper, path, sheet_name):
    sheet_df, dfs = extract_dfs(path, sheet_name)

    last_dates = get_last_dates(sheet_df)
    type_codes = get_type_codes(sheet_df)
    sample_types = get_sample_type(sheet_df)
    label_ids = get_label_ids(type_codes)
    site_ids = get_site_ids(label_ids)

    for v_df, last_date, site_id, sample_type in zip(dfs, last_dates, site_ids, sample_types):
        mapper = apply_quality_checks(mapper, v_df, last_date, site_id, sample_type)
    return mapper

read_validation(qc_lab, QC_VIRUS_DATA, "QC_Compil_STEP (int)")

TypeError: read_excel() got an unexpected keyword argument 'sorted'